# Hessian functions

Created 03/07/2024

Objectives:
* Write functions to calculate hessians of loss functions

# Package imports

In [1]:
import numpy as np

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
import sys
sys.path.append('../')

In [4]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries
from SPTOptimization.utils import (
    get_transfer_matrix_from_unitary,
    multiply_transfer_matrices,
    get_right_identity_environment
)
from SPTOptimization.gradients import expectation_gradients

# Load data

In [5]:
DATA_DIR = r"../data/transverse_cluster_200_site_dmrg/0_50.h5"

In [6]:
with h5py.File(DATA_DIR, 'r') as f:
    data = hdf5_io.load_from_hdf5(f)

In [7]:
psi = data['wavefunction']

# Definitions

In [8]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [9]:
non_trivial_hermitians = np.array([
    1j*np_X,
    1j*np_Y,
    1j*np_Z
])

In [10]:
non_trivial_hermitians.shape

(3, 2, 2)

In [11]:
test = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [np_Y, np_Z, np_X]
)

In [12]:
test.compute_expectation()

array(0.)

In [13]:
test.compute_svd_approximate_expectation()

1.1387329967158467e-17j

In [14]:
test.right_projected_symmetry_state

<npc.Array shape=(8, 8) labels=['vR', 'vR*']>

# Code

## Numpy reference updating

In [15]:
X = np.identity(5)

In [16]:
X

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [17]:
a = X[2:, 2:]

In [18]:
a

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [19]:
a[1,1] = 5

In [20]:
a

array([[1., 0., 0.],
       [0., 5., 0.],
       [0., 0., 1.]])

In [21]:
X

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 5., 0.],
       [0., 0., 0., 0., 1.]])

## recursive expectation hessian function

In [22]:
from SPTOptimization.gradients import expectation_gradient_from_environments

In [23]:
expectation_gradient_from_environments(psi, 100)

<npc.Array shape=(2, 2) labels=['p', 'p*']>

In [24]:
help(enumerate)

Help on class enumerate in module builtins:

class enumerate(object)
 |  enumerate(iterable, start=0)
 |  
 |  Return an enumerate object.
 |  
 |    iterable
 |      an object supporting iteration
 |  
 |  The enumerate object yields pairs containing a count (from start, which
 |  defaults to zero) and a value yielded by the iterable argument.
 |  
 |  enumerate is useful for obtaining an indexed list:
 |      (0, seq[0]), (1, seq[1]), (2, seq[2]), ...
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  __class_getitem__(...)
 |      See PEP 585
 |  
 |  ----------------------------------------------------------------------
 |  

In [25]:
def recursive_expectation_hessian(psi, left_site_index, left_environment,
    right_environment, unitaries, perturbations, transfer_matrices,
    hessian_array, mps_form):

    head_unitary, *tail_unitaries = unitaries
    head_perturbations, *tail_perturbations = perturbations
    head_tm, *tail_tms = transfer_matrices

    perturbation_tms = [
        get_transfer_matrix_from_unitary(
            psi, left_site_index, u, form=mps_form
        )
        for u in head_perturbations
    ]

    new_left_environments = [
        multiply_transfer_matrices(left_environment, tm)
        for tm in perturbation_tms
    ]

    tail_gradients = [
        expectation_gradients(
            psi, tail_tms, left_site_index+1, le, right_environment, mps_form
        )
        for le in new_left_environments
    ]

    for i, gradients in enumerate(tail_gradients):
        for j, (gradient, perturbations) in enumerate(zip(gradients, tail_perturbations), start=1):
            for k, perturbation in enumerate(perturbations):
                np_grad = gradient.to_ndarray()
                out = np.sum(np_grad*perturbation)
                hessian_array[0, i, j, k] = out
                hessian_array[j, k, 0, i] = out

    new_left_environment = multiply_transfer_matrices(
        left_environment, head_tm
    )

    if len(tail_unitaries) > 1:
        recursive_expectation_hessian(
            psi,
            left_site_index+1,
            new_left_environment,
            right_environment,
            tail_unitaries,
            tail_perturbations,
            tail_tms,
            hessian_array[1:, :, 1:, :],
            mps_form
        )

Be careful with how "hessian_array" is interpreted...

In [26]:
def get_expectation_hessian(psi, left_site_index, left_environment,
    right_environment, unitaries, transfer_matrices,
    mps_form):

    n = len(unitaries)

    hessian_array = np.zeros((n, 3, n, 3), dtype=complex)

    perturbations = [np.dot(non_trivial_hermitians, u) for u in unitaries]
    
    recursive_expectation_hessian(
        psi,
        left_site_index,
        left_environment,
        right_environment,
        unitaries,
        perturbations,
        transfer_matrices,
        hessian_array,
        mps_form
    )

    return hessian_array

### Testing

In [27]:
test

In [55]:
h = get_expectation_hessian(
    psi,
    test.right_symmetry_index + 1,
    test.right_projected_symmetry_state,
    get_right_identity_environment(psi, test.right_symmetry_index + 3),
    test.right_boundary_unitaries,
    test.right_transfer_matrices,
    mps_form="B"
)

In [56]:
h

array([[[[ 0.00000000e+00+0.00000000e+00j,
           0.00000000e+00+0.00000000e+00j,
           0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00-2.90532301e-19j,
          -1.56357962e-15+0.00000000e+00j,
           0.00000000e+00-9.49596969e-15j],
         [ 0.00000000e+00+1.48966001e-15j,
          -1.51557505e-14+0.00000000e+00j,
           0.00000000e+00+2.03287907e-20j]],

        [[ 0.00000000e+00+0.00000000e+00j,
           0.00000000e+00+0.00000000e+00j,
           0.00000000e+00+0.00000000e+00j],
         [-1.73472348e-18+0.00000000e+00j,
           0.00000000e+00-1.18391248e-08j,
          -4.57724219e-08+0.00000000e+00j],
         [-3.53098077e-01+0.00000000e+00j,
           0.00000000e+00-6.90136837e-12j,
          -2.77555756e-17+0.00000000e+00j]],

        [[ 0.00000000e+00+0.00000000e+00j,
           0.00000000e+00+0.00000000e+00j,
           0.00000000e+00+0.00000000e+00j],
         [ 0.00000000e+00-1.38777878e-17j,
          -4.20946981e-08+0.00000000e+00j,


In [57]:
h.shape

(3, 3, 3, 3)

In [58]:
np.max(h)

(1.2753451457354705+0j)

In [59]:
np.argmax(h)

34

In [60]:
matrix_h = h.reshape((9,9))

In [61]:
matrix_h

array([[ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00-2.90532301e-19j,
        -1.56357962e-15+0.00000000e+00j,  0.00000000e+00-9.49596969e-15j,
         0.00000000e+00+1.48966001e-15j, -1.51557505e-14+0.00000000e+00j,
         0.00000000e+00+2.03287907e-20j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, -1.73472348e-18+0.00000000e+00j,
         0.00000000e+00-1.18391248e-08j, -4.57724219e-08+0.00000000e+00j,
        -3.53098077e-01+0.00000000e+00j,  0.00000000e+00-6.90136837e-12j,
        -2.77555756e-17+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00-1.38777878e-17j,
        -4.20946981e-08+0.00000000e+00j,  0.00000000e+00+1.62746424e-07j,
         0.00000000e+00+1.36514717e+00j, -2.76622059e-11+0.00000000e+00j,
         0.00000000e+00+2.77

In [62]:
np.max(matrix_h - matrix_h.T)

0j

In [63]:
matrix_h[0,0]

0j

In [64]:
np.max(np.diagonal(matrix_h))

0j

In [65]:
h[0, :, 0, :]

array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]])

In [66]:
h[1, :, 1, :]

array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]])

In [67]:
h[2, :, 2, :]

array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]])

In [68]:
h[0, :, 2, :]

array([[ 0.00000000e+00+1.48966001e-15j, -1.51557505e-14+0.00000000e+00j,
         0.00000000e+00+2.03287907e-20j],
       [-3.53098077e-01+0.00000000e+00j,  0.00000000e+00-6.90136837e-12j,
        -2.77555756e-17+0.00000000e+00j],
       [ 0.00000000e+00+1.36514717e+00j, -2.76622059e-11+0.00000000e+00j,
         0.00000000e+00+2.77555756e-17j]])

Compute expectations

In [69]:
rescale_factor = (
    test.left_expectation
    * test.symmetry_transfer_matrix_singular_vals[0]
)

In [70]:
perturbation1 = np.dot(non_trivial_hermitians[2], np_Y)
perturbation2 = np.dot(non_trivial_hermitians[0], np_X)

In [71]:
test2 = SymmetryActionWithBoundaryUnitaries(
    psi,
    [np_X, np_I]*50,
    left_boundary_unitaries = [np_Z,],
    right_boundary_unitaries = [perturbation1, np_Z, perturbation2]
)

In [72]:
test2.compute_expectation()

array(0.+0.93061628j)

In [73]:
rescale_factor * h[0, 2, 2, 0]

0.930616280444522j

In [74]:
test_h = np.zeros((3,3), dtype=complex)

In [82]:
for i, m1 in enumerate(non_trivial_hermitians):
    for j, m2 in enumerate(non_trivial_hermitians):
        p1 = np.dot(m1, np_Y)
        p2 = np.dot(m2, np_X)

        e = SymmetryActionWithBoundaryUnitaries(
            psi,
            [np_X, np_I]*50,
            left_boundary_unitaries = [np_Z,],
            right_boundary_unitaries = [p1, np_Z, p2]
        )
        
        test_h[i, j] = e.compute_expectation()

In [83]:
test_h

array([[ 0.00000000e+00+0.00000000e+00j, -1.03311743e-14+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [-2.40705783e-01+0.00000000e+00j,  0.00000000e+00-4.70463983e-12j,
        -5.63872175e-18+0.00000000e+00j],
       [ 0.00000000e+00+9.30616280e-01j, -1.88572420e-11+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j]])

In [84]:
rescale_factor * h[0, :, 2, :]

array([[ 0.00000000e+00+1.01549627e-15j, -1.03316247e-14+0.00000000e+00j,
         0.00000000e+00+1.38580689e-20j],
       [-2.40705783e-01+0.00000000e+00j,  0.00000000e+00-4.70463983e-12j,
        -1.89208835e-17+0.00000000e+00j],
       [ 0.00000000e+00+9.30616280e-01j, -1.88572336e-11+0.00000000e+00j,
         0.00000000e+00+1.89208835e-17j]])

In [85]:
np.max(np.abs(test_h - rescale_factor * h[0, :, 2, :]))

1.0154962677034446e-15

In [95]:
for i, m1 in enumerate(non_trivial_hermitians):
    for j, m2 in enumerate(non_trivial_hermitians):
        p1 = np.dot(m1, np_Y)
        p2 = np.dot(m2, np_Z)

        e = SymmetryActionWithBoundaryUnitaries(
            psi,
            [np_X, np_I]*50,
            left_boundary_unitaries = [np_Z,],
            right_boundary_unitaries = [p1, p2, np_X]
        )
        
        test_h[i, j] = e.compute_expectation()

In [96]:
test_h

array([[ 0.00000000e+00+0.00000000e+00j, -1.06324967e-15+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [-3.87950843e-19+0.00000000e+00j,  0.00000000e+00-8.07069194e-09j,
        -3.12029076e-08+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j, -2.86958155e-08+0.00000000e+00j,
         0.00000000e+00+1.10943695e-07j]])

In [97]:
rescale_factor * h[0, :, 1, :]

array([[ 0.00000000e+00-1.98054902e-19j, -1.06588702e-15+0.00000000e+00j,
         0.00000000e+00-6.47337092e-15j],
       [-1.18255522e-18+0.00000000e+00j,  0.00000000e+00-8.07069195e-09j,
        -3.12029076e-08+0.00000000e+00j],
       [ 0.00000000e+00-9.46044173e-18j, -2.86958155e-08+0.00000000e+00j,
         0.00000000e+00+1.10943695e-07j]])

In [98]:
np.max(np.abs(test_h - rescale_factor * h[0, :, 1, :]))

6.473370916506173e-15

In [90]:
for i, m1 in enumerate(non_trivial_hermitians):
    for j, m2 in enumerate(non_trivial_hermitians):
        p1 = np.dot(m1, np_Z)
        p2 = np.dot(m2, np_X)

        e = SymmetryActionWithBoundaryUnitaries(
            psi,
            [np_X, np_I]*50,
            left_boundary_unitaries = [np_Z,],
            right_boundary_unitaries = [np_Y, p1, p2]
        )
        
        test_h[i, j] = e.compute_expectation()

In [91]:
test_h

array([[ 0.00000000e+00-1.88608753e-11j,  8.69398539e-01+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [-2.42348324e-17+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -9.70757050e-08+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j, -1.82113166e-17+0.00000000e+00j,
         0.00000000e+00+2.51088338e-08j]])

In [92]:
rescale_factor * h[1, :, 2, :]

array([[ 0.00000000e+00-1.88608664e-11j,  8.69398539e-01+0.00000000e+00j,
         0.00000000e+00-2.08633228e-17j],
       [ 4.39653629e-17+0.00000000e+00j,  0.00000000e+00-3.65943289e-17j,
        -9.70757049e-08+0.00000000e+00j],
       [ 0.00000000e+00+4.74168333e-18j, -2.09211241e-17+0.00000000e+00j,
         0.00000000e+00+2.51088338e-08j]])

In [94]:
np.max(np.abs(test_h - rescale_factor * h[1, :, 2, :]))

9.992007221626409e-16

Looks good!

In [99]:
def get_expectation_gradient_fixed_basis(expectation_gradients, unitaries):
    gradients = np.array([g.to_ndarray() for g in expectation_gradients])
    perturbations = np.array([np.dot(non_trivial_hermitians, u) for u in unitaries])

    perturbation_gradients = np.sum(
        gradients[:, np.newaxis, ...] * perturbations,
        (-1, -2)
    )

    return perturbation_gradients

In [100]:
gradients = expectation_gradients(
    psi,
    test.right_transfer_matrices,
    test.right_symmetry_index+1,
    test.right_projected_symmetry_state
)

In [101]:
gradients[0]

<npc.Array shape=(2, 2) labels=['p*', 'p']>

In [102]:
X = get_expectation_gradient_fixed_basis(gradients, test.right_boundary_unitaries)

In [104]:
X.shape

(3, 3)

In [105]:
def get_quadratic_expectation_hessian(expectation, expectation_gradients,
                                      expectation_hessian):

    n = expectation_hessian.shape[0]*expectation_hessian.shape[1]

    flat_exp_hessian = np.reshape(expectation_hessian, (n,n))
    h1 = np.real(expectation*np.conj(flat_exp_hessian))

    flat_exp_grads = np.reshape(expectation_gradients, (n))

    second_order_from_first_hessian = (
        flat_exp_grads[:, np.newaxis] + flat_exp_grads[np.newaxis, :]
    )

    h2 = np.abs(second_order_from_first_hessian)**2

    return 2*(h1 + h2)

In [106]:
full_h = get_quadratic_expectation_hessian(
        test.expectation,
        X,
        h
    )

In [107]:
full_h.shape

(9, 9)

In [108]:
full_h

array([[3.52702330e-27, 3.59068443e-02, 2.49356504e-01, 9.52478914e-23,
        8.81755952e-28, 8.82338708e-28, 8.77026007e-28, 8.81755825e-28,
        1.49419982e-27],
       [3.59068443e-02, 1.43627377e-01, 2.85263348e-01, 3.59068443e-02,
        3.59068443e-02, 3.59068443e-02, 3.59068443e-02, 3.59068443e-02,
        3.59068443e-02],
       [2.49356504e-01, 2.85263348e-01, 9.97426015e-01, 2.49356504e-01,
        2.49356504e-01, 2.49356504e-01, 2.49356504e-01, 2.49356504e-01,
        2.49356504e-01],
       [9.52478914e-23, 3.59068443e-02, 2.49356504e-01, 3.83313511e-22,
        9.58283778e-23, 9.58281857e-23, 9.58299392e-23, 9.58283778e-23,
        9.56530257e-23],
       [8.81755952e-28, 3.59068443e-02, 2.49356504e-01, 9.58283778e-23,
        5.06298641e-34, 2.22871158e-34, 6.48643552e-33, 1.28144149e-34,
        8.02908178e-29],
       [8.82338708e-28, 3.59068443e-02, 2.49356504e-01, 9.58281857e-23,
        2.22871158e-34, 3.85185989e-34, 4.89099625e-33, 9.63013326e-35,
        8.0

In [110]:
np.linalg.cholesky(full_h)

LinAlgError: Matrix is not positive definite

In [111]:
l = np.linalg.eigvalsh(full_h)

In [112]:
l

array([-3.32490283e-01, -5.26609219e-18, -2.17188302e-18, -1.43784166e-19,
       -4.81482486e-35,  8.60744946e-19,  4.76945052e-18,  7.70089597e-02,
        1.39653472e+00])